# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [8]:
# !nvidia-smi

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace**


### Download train/test metadata

In [9]:
# !pip install --upgrade gdown

# # Main link
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# # Backup link 1
# # !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# # Bqckup link 2
# # !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# !unzip -q libriphone.zip
# !ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [10]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm
from torchensemble import VotingClassifier

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in enumerate(usage_list):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]
    X = X.reshape(-1,concat_nframes,39)
    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X


## Define Dataset

In [11]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# LSTM
class LSTM(nn.Module):
    # input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim
    def __init__(self, input_dim, hidden_dim, hidden_layers, num_class=41):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=hidden_layers,
            dropout = 0.5,
            bidirectional=True,
            batch_first = True,
        )
        self.act = nn.ReLU()
        self.drop = nn.Dropout(p=0.5),
        self.fc = nn.Linear(hidden_dim*2, num_class)

    def forward(self, inputs):
        out, (h_n, c_n) = self.lstm(inputs, None)
        out = self.act(out[:,5,:])
        out = self.fc(out)

        return out

## Hyper-parameters

In [18]:
# data prarameters
concat_nframes = 47              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 1               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 5098                        # random seed
batch_size = 256                # batch size
num_epoch = 10                   # the number of training epoch
learning_rate = 0.001          # learning rate
model_path = './modelv2.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 # the input dim of the model, you should not change the value
hidden_layers = 4               # the number of hidden layers
hidden_dim = 128                # the hidden dim

## Prepare dataset and model

In [19]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 4286
[INFO] train set
torch.Size([2644158, 47, 39])
torch.Size([2644158])
[Dataset] - # phone classes: 41, number of utterances for val: 0
[INFO] val set
torch.Size([0, 47, 39])
torch.Size([0])


In [20]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')
torch.cuda.set_device(2)
print(torch.cuda.current_device())

DEVICE: cuda:2
2


In [21]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [22]:

# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = VotingClassifier(
    estimator=LSTM(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim),
    n_estimators=6,
    cuda=True,
)
model.set_optimizer("RMSprop", lr=learning_rate)
model.fit(train_loader, epochs=num_epoch)

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 3.71994 | Correct: 3/256
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 2.73511 | Correct: 65/256
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 2.50092 | Correct: 86/256
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 2.25398 | Correct: 90/256
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 2.20239 | Correct: 99/256
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 2.02101 | Correct: 110/256
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.98535 | Correct: 102/256
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 1.76334 | Correct: 129/256
Estimator: 000 | Epoch: 000 | Batch: 800 | Loss: 1.86107 | Correct: 126/256
Estimator: 000 | Epoch: 000 | Batch: 900 | Loss: 1.79810 | Correct: 120/256
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 1.62465 | Correct: 138/256
Estimator: 000 | Epoch: 000 | Batch: 1100 | Loss: 1.53442 | Correct: 132/256
Estimator: 000 | Epoch: 000 | Batch: 1200 | Loss: 1.43711 | Correct: 145/256
Estimator: 000 

## Training

In [23]:
del train_loader, val_loader
gc.collect()

24

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [24]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078
[INFO] test set
torch.Size([646268, 47, 39])


In [25]:
# load model
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
# model = LSTM(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
# model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
# test_acc = 0.0
# test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        outputs = model.predict(batch)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('ensemblev2.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))